In [44]:
import preprocessing

import os
import sys
import pickle
import numpy as np
import pandas as pd
import keras
import h5py

from keras.utils import to_categorical
# from keras.layers import GRU, LSTM, Dropout, Dense, Input, TimeDistributed, Activation, Flatten, Concatenate
from keras.layers import *
from keras.models import Model, Sequential
from keras.models import load_model
# from keras.callbacks import *

import tensorflow as tf

def init():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    keras.backend.tensorflow_backend.set_session(session)

init()

In [46]:
n_user_train = 462
n_user_test = 74
n_sen_train = 1716
n_sen_test = 342

path_data = 'data/'
mfcc_or_fbank = 'mfcc'
model_name = 'RNN'
GL = 'LSTM' # GRU or LSTM

if mfcc_or_fbank == 'mfcc' :
    dim = 39
else :
    dim = 69

# RNN seting
n_RNN_seq = 3

if model_name == 'RNN' :
    n_seq = n_RNN_seq
    
# for traingin 
batch_size = 1024

# for pred_to_ans
size_window = 7

In [8]:
#
# RNN model
#
def RNN_model() :
    dr_r = 0.5
    I = Input(shape=((n_seq,dim))) # shape = (?,1,200,2)
#     gru1 = GRU(32, activation='relu', dropout=0.0, return_sequences=True)(I)
#     gru2 = GRU(32, activation='relu', dropout=0.0, return_sequences=True)(gru1)
#     gru3 = GRU(64, activation='relu', dropout=0.0, return_sequences=True)(gru2)
#     gru4 = GRU(64, activation='relu', dropout=0.0, return_sequences=True)(gru3)
    
#     gru12 = GRU(32, activation='relu', dropout=0.0, return_sequences=True, go_backwards=True)(I)
#     gru22 = GRU(32, activation='relu', dropout=0.0, return_sequences=True, go_backwards=True)(gru12)
#     gru32 = GRU(64, activation='relu', dropout=0.0, return_sequences=True, go_backwards=True)(gru22)
#     gru42 = GRU(64, activation='relu', dropout=0.0, return_sequences=True, go_backwards=True)(gru32)
    
#     F1 = Flatten()(gru4)
#     F2 = Flatten()(gru42)
#     C1 = Concatenate()([F1,F2])
#     Dr1 = Dropout(dr_r)(C1)

    if GL == 'LSTM' :
        B1 = wrappers.Bidirectional(LSTM(64, activation='elu', dropout=0.0, return_sequences=True), merge_mode='concat', weights=None)(I)
        B2 = wrappers.Bidirectional(LSTM(128, activation='elu', dropout=0.0, return_sequences=True), merge_mode='concat', weights=None)(B1)
        B3 = wrappers.Bidirectional(LSTM(128, activation='elu', dropout=0.0, return_sequences=True), merge_mode='concat', weights=None)(B2)
        B4 = wrappers.Bidirectional(LSTM(64, activation='elu', dropout=dr_r, return_sequences=True), merge_mode='concat', weights=None)(B3)
    elif GL == 'GRU' :
        B1 = wrappers.Bidirectional(GRU(64, activation='elu', dropout=0.0, return_sequences=True), merge_mode='concat', weights=None)(I)
        B2 = wrappers.Bidirectional(GRU(128, activation='elu', dropout=0.0, return_sequences=True), merge_mode='concat', weights=None)(B1)
        B3 = wrappers.Bidirectional(GRU(128, activation='elu', dropout=0.0, return_sequences=True), merge_mode='concat', weights=None)(B2)
        B4 = wrappers.Bidirectional(GRU(64, activation='elu', dropout=dr_r, return_sequences=True), merge_mode='concat', weights=None)(B3)
    
    gru100 = GRU(48, activation='softmax', dropout=0.0, return_sequences=True)(B4)

    model = Model(I,gru100)
    model.compile(#loss='mean_squared_error',
                      loss='categorical_crossentropy',
                      #loss='sparse_categorical_crossentropy',
                      optimizer='rmsprop',
                      #optimizer='adam',
                      #optimizer='sgd',
                      metrics=['acc']) #'mae'
    print (model.summary())
    return model

In [9]:
#
# loading data
#
'{}data_pp/X_train_{}_{}_{}.npy'.format(path_data, model_name, mfcc_or_fbank, n_seq)
X_train = np.load('{}data_pp/X_train_{}_{}_{}.npy'.format(path_data, model_name, mfcc_or_fbank, n_seq))
y_train = np.load('{}data_pp/y_train_{}_{}_{}.npy'.format(path_data, model_name, mfcc_or_fbank, n_seq))
X_test = np.load('{}data_pp/X_test_{}_{}_{}.npy'.format(path_data, model_name, mfcc_or_fbank, n_seq))

In [11]:
y_train = y_train.reshape((-1))
y_train_dummy = to_categorical(y_train, num_classes=48)
y_train_dummy = y_train_dummy.reshape((-1,3,48))
print (X_train.shape)
print (y_train_dummy.shape)

(1117431, 3, 39)
(1117431, 3, 48)


In [12]:
if not os.path.isdir('{}model'.format(path_data)) :
    os.mkdir('{}model'.format(path_data))

MCP = keras.callbacks.ModelCheckpoint('{}model/{}_{}_{}_{}.h5'.format(path_data, model_name, mfcc_or_fbank, n_seq, GL), monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
ES = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=0, mode='auto')
RM = keras.callbacks.RemoteMonitor(root='http://localhost:12333', path='/publish/epoch/end/', field='data', headers=None)

model = RNN_model()
model.fit(X_train, y_train_dummy, epochs=50, batch_size=batch_size, validation_split=0.1, callbacks=[MCP,ES,RM])

model = load_model('{}model/{}_{}_{}_{}.h5'.format(path_data, model_name, mfcc_or_fbank, n_seq, GL))

pred = model.predict(X_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 39)             0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 3, 128)            39936     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 3, 128)            74112     
_________________________________________________________________
gru_5 (GRU)                  (None, 3, 48)             25488     
Total params: 139,536
Trainable params: 139,536
Non-trainable params: 0
_________________________________________________________________
None
Train on 838073 samples, validate on 279358 samples
Epoch 1/1
838073/838073 [==============================] - 25s - loss: 1.6485 - acc: 0.5170 - val_loss: 1.2648 - val_acc: 0.6018

In [20]:
print (pred.shape)

(179222, 3, 48)
[[  1.28994070e-05   3.63423096e-05   4.79725917e-04   0.00000000e+00
    4.46228478e-06   0.00000000e+00   6.18973354e-05   4.29209234e-04
    4.10770263e-05   0.00000000e+00   6.89661014e-04   1.99446711e-03
    0.00000000e+00   3.32118452e-06   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.52108725e-04   8.41366855e-05   1.12028851e-03
    7.56452646e-05   1.14374631e-03   3.02499742e-04   0.00000000e+00
    7.53876520e-04   2.67336640e-04   9.10397328e-04   1.00933644e-03
    3.03840876e-04   1.30459631e-03   2.85098067e-04   1.59808151e-05
    0.00000000e+00   1.30434241e-03   9.46857836e-05   6.18919916e-03
    3.84060229e-04   6.76554382e-01   8.98951665e-04   4.02151421e-03
    0.00000000e+00   2.37977019e-05   9.50443849e-04   0.00000000e+00
    3.80227226e-04   0.00000000e+00   2.17110966e-03   0.00000000e+00]
 [  1.30821372e-05   2.63942475e-05   4.79362556e-04   0.00000000e+00
    4.43781119e-06   0.00000000e+00   5.68256801e-05   4.72412561e-04
   

In [53]:
def predict_to_ans(ary_pred) :
    def num_to_char(ary_pred_num) :
        map_48phone_char = pd.read_csv('{}48phone_char.map'.format(path_data), header=None, delimiter='\t')
        dict_map_48phone_char = dict()
        for row in map_48phone_char.iterrows() :
            #dict_map_48char[name[1][0]] = dict_map_39char[name[1][1]]
            dict_map_48phone_char[row[1][1]] = row[1][2]
        flat = ary_pred_num.flatten()
        flat_copy = flat.copy().astype(str)
        for i,num in enumerate(flat) :
            flat_copy[i] = dict_map_48phone_char[num]
        ary_pred_char = flat_copy.reshape((-1,n_seq))
        return ary_pred_char

    def find_mass_row(string) :
        dict_count = dict()
        for item in string :
            try :
                dict_count[item] += 1
            except :
                dict_count[item] = 1
        max_item = max(dict_count, key=dict_count.get)
        return max_item
    
    def find_mass_column(ary_pred, i_data_total) :
        dict_count = dict()
        for i in range(n_seq) :
            item = ary_pred[i_data_total-n_seq+1+i][n_seq-1-i]
            try :
                dict_count[item] += 1
            except :
                dict_count[item] = 1
        max_item = max(dict_count, key=dict_count.get)
        return max_item
    
    def to_str_final(string) :
        #
        # combine two char if they are same
        #
        str_final = string[0]
        for c in string[1:] :
            if c != str_final[-1] :
                str_final += c
        #
        # cut all sil in the begining
        #
        if str_final[0] == 'L' :
            str_final = str_final[1:]
            
        return str_final
    
    df_BE_test = pd.read_csv('{}data_pp/beginEnd_test.csv'.format(path_data))
    
    ary_pred_num = np.argmax(ary_pred, axis=2)
    print (ary_pred_num[200:206])
    ary_pred_char = num_to_char(ary_pred_num)
    
    lst_X_data = []
    i_data_total = 0 # for loop use
    ans = []
    for BE in df_BE_test.iterrows() :
        index_begin = BE[1]['index_begin']
        index_end = BE[1]['index_end']
        length_BE = BE[1]['length']
        n_data = length_BE - n_seq + 1
        assert n_data >= 1, 'n_data should bigger than 1, please do checking'
        
        str_temp = ''
        str_temp_2 = ''
        for i_data in range(n_data) :
            dic_temp = dict()
            if (i_data < n_seq - 1) : # don't count the begining and end sequence
                i_data_total += 1
                continue
            else :
                str_temp += str(find_mass_column(ary_pred_char, i_data_total))
                i_data_total += 1
        assert len(str_temp) == n_data - n_seq + 1, 'len(str_temp) != n_data - n_seq + 1, please check'
        
        for t in range(len(str_temp) - size_window + 1) :
            str_temp_2 += str(find_mass_row(str_temp[t:t+size_window]))
        assert len(str_temp_2) == len(str_temp) - size_window + 1, 'len(str_temp_2) != len(str_temp) - size_window + 1, please check'
        
        str_final = to_str_final(str_temp_2)
        
        ans += [str_final]
        
    print ('max_len_ans : {}'.format(max(len(x) for x in ans)))
    
    sample = pd.read_csv('{}sample.csv'.format(path_data))
    assert len(sample['phone_sequence']) == len(ans), 'len(sample[\'phone_sequence\']) != len(ans), please check'
    
    sample['phone_sequence'] = pd.DataFrame(ans)
    if not os.path.isdir('./ans') :
        os.mkdir('./ans')
    sample.to_csv('./ans/ans_{}_{}_{}_{}.csv'.format(model_name, mfcc_or_fbank, n_seq, GL), index=False)
        
    return sample
    
ans = predict_to_ans(pred)

[[19 19 19]
 [19 19 19]
 [33 33 13]
 [33 13  1]
 [13 13 13]
 [13 13 13]]
max_len_ans : 96


In [52]:
print (ans[:5])

             id                                     phone_sequence
0  fadg0_si1279                             HIrLAJUarDaBLMrDwLMwtL
1  fadg0_si1909                    tnbLKwnwLywDBwJIrBJLhwLDyLtwKrL
2   fadg0_si649  wLJcnctrJtrDBgbLAwLKTyDwLMJywLHrLlwJwDIrJLhnLM...
3   fadg0_sx109           iKyJBwLMwDJLHrIsEDIrIbsLABaDwJLMyLJBFLHL
4    fadg0_sx19                  AaBFDnDUJFBSrLAwDaBtwDhytBcDUJwyL
